In [9]:
import numpy as np
from pyqubo import Binary, Array

In [10]:
N = 4
K = 2

In [20]:
# Generate randomized and valid distance matrix
np.random.seed(
    pow(hash("iQuHack 2021"), 2, 2**32)
)
random_matrix = np.random.rand(N, N)
symm_matrix = (random_matrix + random_matrix.T)/2
np.fill_diagonal(symm_matrix, 0)
dist_matrix = symm_matrix
dist_matrix

array([[0.        , 0.69094463, 0.89051795, 0.62171336],
       [0.69094463, 0.        , 0.98001849, 0.4567722 ],
       [0.89051795, 0.98001849, 0.        , 0.47575806],
       [0.62171336, 0.4567722 , 0.47575806, 0.        ]])

In [42]:
# x_{i, j} = point i belongs to cluster j
group_matrix = Array.create('x', shape=(N, K), vartype='BINARY')
group_matrix

Array([[Binary(x[0][0]), Binary(x[0][1])],
       [Binary(x[1][0]), Binary(x[1][1])],
       [Binary(x[2][0]), Binary(x[2][1])],
       [Binary(x[3][0]), Binary(x[3][1])]])

In [30]:
distinct_size_penalty = 1
multiple_assignment_penalty = 1000

In [31]:
all_terms = []

In [40]:
# Penalty for being assigned to multiple clusters
for i in range(N):
    single_asignment_constraint = (1 - sum(group_matrix[i,:]))**2
    all_terms.append(
        single_asignment_constraint * multiple_assignment_penalty
    )

In [43]:
# Penalty for exceding equal number of members per cluster
for j in range(K):
    target_size = N//K
    cluster_member_constraint = (target_size - sum(group_matrix[:, j]))**2
    all_terms.append(
        multiple_assignment_penalty*cluster_member_constraint
    )

In [51]:
# Adding objective for distances. Must minimize distances in the same group
for i in range(N):
    for j in range(i+1, N):
        same_group_combinations = group_matrix[i,:] * group_matrix[j,:]
        all_terms.append(
            dist_matrix[i,j] * sum(same_group_combinations)
        )


In [52]:
equation = sum(all_terms)
equation.compile().to_qubo()

({('x[1][0]', 'x[1][0]'): -4000.0,
  ('x[2][1]', 'x[0][1]'): 2001.7810359060152,
  ('x[1][0]', 'x[1][1]'): 2000.0,
  ('x[3][0]', 'x[0][0]'): 2001.243426723804,
  ('x[2][0]', 'x[0][0]'): 2001.7810359060152,
  ('x[3][1]', 'x[1][1]'): 2000.9135444032406,
  ('x[2][0]', 'x[2][1]'): 2000.0,
  ('x[1][0]', 'x[0][0]'): 2001.3818892643808,
  ('x[2][0]', 'x[3][0]'): 2000.951516113187,
  ('x[2][0]', 'x[1][0]'): 2001.9600369896034,
  ('x[3][0]', 'x[3][0]'): -4000.0,
  ('x[0][0]', 'x[0][0]'): -4000.0,
  ('x[2][1]', 'x[1][1]'): 2001.9600369896034,
  ('x[3][0]', 'x[1][0]'): 2000.9135444032406,
  ('x[3][0]', 'x[3][1]'): 2000.0,
  ('x[1][1]', 'x[0][1]'): 2001.3818892643808,
  ('x[3][1]', 'x[0][1]'): 2001.243426723804,
  ('x[2][1]', 'x[3][1]'): 2000.951516113187,
  ('x[0][0]', 'x[0][1]'): 2000.0,
  ('x[1][1]', 'x[1][1]'): -4000.0,
  ('x[3][1]', 'x[3][1]'): -4000.0,
  ('x[2][0]', 'x[2][0]'): -4000.0,
  ('x[2][1]', 'x[2][1]'): -4000.0,
  ('x[0][1]', 'x[0][1]'): -4000.0},
 12000.0)